In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from porespy import generators
from tomo_encoders.misc_utils.feature_maps_vis import view_midplanes
import cupy as cp
import time
import h5py
from tomopy import project
sys.path.append('/data02/MyArchive/aisteer_3Dencoders/TomoEncoders/tomo_encoders/tasks/sparse_segmenter/')
from recon_subvol import fbp_filter, recon_patch#, recon_patches_3d
from tomo_encoders import DataFile, Patches
import os
import signal
import tomocg as pt

In [2]:
# # Use managed memory
# cp.cuda.set_allocator(cp.cuda.MemoryPool(cp.cuda.malloc_managed).malloc)

### Read projections data  

In [3]:
filename = 'mli_L206_HT_650_L3'
projs_path = '/data02/MyArchive/AM_part_Xuan/projs' 
binning = 1 
ntheta = 3000

In [4]:
read_fpath = os.path.join(projs_path, filename + '_projs_bin%i_ntheta%i.hdf5'%(binning,ntheta))
with h5py.File(read_fpath, 'r') as hf:
    projs = np.asarray(hf['data'][:])
    theta = np.asarray(hf['theta'][:])
    center = float(np.asarray(hf['center'][()]))

In [5]:
# print("projections shape: %s"%str(projs.shape))
# fig, ax = plt.subplots(1,1, figsize = (16,8))
# ax.imshow(projs[0], cmap = 'gray')

In [6]:
# fig, ax = plt.subplots(1,1, figsize = (16,8))
# ax.imshow(projs[:,500,:], cmap = 'gray')

## Reconstruct one full slice  

In [7]:
vol_shape = projs.shape[1:] + (projs.shape[-1],)
vol_shape

(1102, 4200, 4200)

In [8]:
model_size = (64,64,64)
kwargs = {"initialize_by" : 'regular-grid',
          "patch_size" : model_size, \
          "stride" : 1}
p_grid = Patches(vol_shape, **kwargs)
print(p_grid.points.shape)
p_grid._check_valid_points()

(71825, 3)


In [9]:
p3d = p_grid.select_random_sample(100)

In [10]:
z_idxs = p3d.points[:,0]
p2d = Patches(tuple(vol_shape[1:]), initialize_by = "data", \
              points = p3d.points[:,1:], \
              widths = p3d.widths[:,1:])
p2d.add_features(z_idxs.reshape(-1,1), names = ["z_idx"])

p2d = p2d.sort_by_feature(ife = 0)
z_idxs_unique = np.unique(z_idxs)

In [11]:
for z_idx in z_idxs_unique:
    p2d_z = p2d.filter_by_condition(p2d.features[:,0] == z_idx)
    print("index %i, number of patches: %i"%(z_idx, len(p2d_z)))

index 0, number of patches: 3
index 64, number of patches: 3
index 128, number of patches: 8
index 192, number of patches: 5
index 256, number of patches: 3
index 320, number of patches: 3
index 384, number of patches: 7
index 448, number of patches: 5
index 512, number of patches: 7
index 576, number of patches: 6
index 640, number of patches: 5
index 704, number of patches: 9
index 768, number of patches: 6
index 832, number of patches: 11
index 896, number of patches: 5
index 960, number of patches: 7
index 1024, number of patches: 7


In [12]:
z_idx = 640
p2d_z = p2d.filter_by_condition(p2d.features[:,0] == z_idx)

In [13]:
len(p2d_z)

5

In [14]:
# with cp.cuda.Device(0):
#     center = projs.shape[-1]//2.0
#     sub_vol = recon_patch(projs, theta, center, point, width, mem_limit_gpu = 40.0, apply_fbp=True)

In [15]:
with cp.cuda.Device(0):
    center = projs.shape[-1]//2.0
    
    sub_vols = recon_patches_3d
    sub_vol = recon_patch(projs, theta, center, point, width, \
                          mem_limit_gpu = 40.0,
                          apply_fbp=True, nzc = 8)
    sub_vol = 1.0*(sub_vol - sub_vol.min()) / (sub_vol.max() - sub_vol.min())

NameError: name 'recon_patches_3d' is not defined

In [ ]:
any([False,False,False])

In [ ]:
center = projs.shape[-1]//2.0


# point = [500//binning, 0, 0]
# point = [0, 0, 0]
# width = [1, 4200//binning, 4200//binning]


# z, y, x
point = [500, 2000//binning, 2000//binning]
width = [128, 256, 256]

# point = [500, 0, 0]
# width = [1, 2100, 2100]


with cp.cuda.Device(0):
    center = projs.shape[-1]//2.0
    sub_vol = recon_patch(projs, theta, center, point, width, \
                          mem_limit_gpu = 40.0,
                          apply_fbp=True, nzc = 8)
    sub_vol = 1.0*(sub_vol - sub_vol.min()) / (sub_vol.max() - sub_vol.min())

In [ ]:
5%2

In [ ]:
sub_vol.shape

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (16,16))
ax.imshow(sub_vol[0], cmap = 'gray')